[![Jupyter Notebook](https://img.shields.io/badge/Jupyter%20Notebook-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/flow.ipynb)

# Validate & register flow cytometry data

In [ ]:
!lamin init --storage ./test-flow --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb
import readfcs

lb.settings.species = "human"  # globally set species

In [ ]:
ln.track()

We start with a flow cytometry file from Alpert19:

In [ ]:
ln.dev.datasets.file_fcs_alpert19()

Use [readfcs](https://lamin.ai/docs/readfcs) to read the fcs file into memory:

In [ ]:
adata = readfcs.read("Alpert19.fcs")
adata

## Track data with cell markers

We'll use the `CellMarker` reference to link features:

In [ ]:
file = ln.File.from_anndata(adata, description="Alpert19", var_ref=lb.CellMarker.name)

We see that many features aren't validated. Let's standardize the identifiers:

In [ ]:
adata.var.index = lb.CellMarker.standardize(adata.var.index)

Now things look much better, but we still have 5 CellMaker records that seem more like metadata.

In [ ]:
file = ln.File.from_anndata(adata, description="Alpert19", var_ref=lb.CellMarker.name)

Hence, let's curate the AnnData a bit more:

In [ ]:
validated = lb.CellMarker.bionty().validate(adata.var.index, "name")

Let's move metadata (non-validated cell markers) into `adata.obs`:

In [ ]:
adata.obs = adata[:, ~validated].to_df()
adata = adata[:, validated].copy()

Now we have a clean panel of 35 CellMarkers and metadata that we don't want to register:

In [ ]:
file = ln.File.from_anndata(adata, description="Alpert19", var_ref=lb.CellMarker.name)

In [ ]:
file.save()

In [ ]:
file.features

In [ ]:
file.features["var"].df().head(10)

Let's register another flow file:

In [ ]:
adata2 = readfcs.read(ln.dev.datasets.file_fcs())
file2 = ln.File.from_anndata(
    adata2, description="My fcs file", var_ref=lb.CellMarker.name
)

In [ ]:
adata2.var.index = lb.CellMarker.standardize(adata2.var.index)

In [ ]:
file2 = ln.File.from_anndata(
    adata2, description="My fcs file", var_ref=lb.CellMarker.name
)

In [ ]:
file2.save()

In [ ]:
file2.view_lineage()

## Query by cell markers

Which datasets have CD14 in the flow panel:

In [ ]:
cell_markers = lb.CellMarker.lookup()

In [ ]:
cell_markers.cd14

In [ ]:
panels_with_cd14 = ln.FeatureSet.filter(cell_markers=cell_markers.cd14).all()

In [ ]:
ln.File.filter(feature_sets__in=panels_with_cd14).df()

Shared cell markers between two files:

In [ ]:
files = ln.File.filter(feature_sets__in=panels_with_cd14).list()
file1, file2 = files[0], files[1]

In [ ]:
file1_markers = file1.features["var"]
file2_markers = file2.features["var"]

shared_markers = file1_markers & file2_markers
shared_markers.list("name")

## Flow marker registry

Check out your CellMarker registry:

In [ ]:
lb.CellMarker.filter().df()

In [ ]:
# a few tests
assert set(shared_markers.list("name")) == set(
    [
        "Ccr7",
        "CD3",
        "Cd14",
        "Cd19",
        "CD127",
        "CD27",
        "CD28",
        "CD8",
        "Cd4",
        "CD57",
    ]
)
ln.File.filter(feature_sets__in=panels_with_cd14).exists()

In [ ]:
# clean up test instance
!lamin delete --force test-flow
!rm -r test-flow